In [22]:
import os

In [26]:
# python (Jupyter) - change to the project folder directly
%cd c:\Users\pc\Desktop\ai\mlops-datascience

c:\Users\pc\Desktop\ai\mlops-datascience


c:\Users\pc\Desktop\ai\mlops-datascience\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:

os.chdir("")


In [27]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [29]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories

In [30]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionconfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        
        return data_ingestion_config

In [31]:
import urllib.request as request
from src.mlops_pipeline import logger
import zipfile

In [32]:
#DataIngestion Component

class DataIngestion:
    def __init__(self,config:DataIngestionconfig):
        self.config=config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"Rile already exists")
            
            
    def extract_zip_file(self):
        """ 
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returne None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [33]:
try:
    config= ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e
    

[2025-11-17 14:03:59,446: INFO: common :yaml file: config\config.yaml loaded]
[2025-11-17 14:03:59,450: INFO: common :yaml file: params.yaml loaded]


ValueError: yaml file is empty

In [21]:
from src.mlops_pipeline.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.mlops_pipeline.utils.common import read_yaml

for p in (CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH):
    try:
        print("OK:", p)
        read_yaml(p)
    except Exception as e:
        print("ERROR:", p, "->", e)

OK: config\config.yaml
[2025-11-17 14:00:57,565: INFO: common :yaml file: config\config.yaml loaded]
OK: params.yaml
[2025-11-17 14:00:57,568: INFO: common :yaml file: params.yaml loaded]
ERROR: params.yaml -> yaml file is empty
OK: schema.yaml
[2025-11-17 14:00:57,585: INFO: common :yaml file: schema.yaml loaded]


In [34]:
from pathlib import Path
import yaml

p = Path("params.yaml").resolve()
print("PATH:", p)
raw = p.read_text(encoding="utf-8")
print("RAW CONTENT:\n", raw)
parsed = yaml.safe_load(raw)
print("PARSED TYPE:", type(parsed))
print("PARSED VALUE:", parsed)

PATH: C:\Users\pc\Desktop\ai\mlops-datascience\params.yaml
RAW CONTENT:
 key:value
PARSED TYPE: <class 'str'>
PARSED VALUE: key:value
